In [ ]:
pip install Bio

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
import os
from Bio import SeqIO
import random
import csv
import re
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
def Z(protein):
    z = {
        "A": [0.07, -1.73, 0.09],
        "V": [-2.69, -2.53, -1.29],
        "L": [-4.19, -1.03, -0.98],
        "I": [-4.44, -1.68, -1.03],
        "P": [-1.22, 0.88, 2.23],
        "F": [-4.92, 1.30, 0.45],
        "W": [-4.75, 3.65, 0.85],
        "M": [-2.49, -0.27, -0.41],
        "K": [2.84, 1.41, -3.14],
        "R": [2.88, 2.52, -3.44],
        "H": [2.41, 1.74, 1.11],
        "G": [2.23, -5.36, 0.30],
        "S": [1.96, -1.63, 0.57],
        "T": [0.92, -2.09, -1.40],
        "C": [0.71, -0.97, 4.13],
        "Y": [-1.39, 2.32, 0.01],
        "N": [3.22, 1.45, 0.84],
        "Q": [2.18, 0.53, -1.14],
        "D": [3.64, 1.13, 2.36],
        "E": [3.08, 0.39, -0.07]
    }

    z_values = []
    for character in protein:
        z_values.append(z.get(character, [0, 0, 0]))

    return z_values

In [ ]:
def ACC(protein):
    ACCN = [0] * 45
    z_values = Z(protein)

    for l in range(1, 6):
        n = len(z_values)
        cal = 0
        for j in range(n - l):
            cal += (z_values[j][0] * z_values[j+1][0] / (n - l))
        ACCN[(l-1)*9] = cal

        cal = 0
        for j in range(n - l):
            cal += (z_values[j][1] * z_values[j+1][1] / (n - l))
        ACCN[(l-1)*9 + 1] = cal

        cal = 0
        for j in range(n - l):
            cal += (z_values[j][2] * z_values[j+1][2] / (n - l))
        ACCN[(l-1)*9 + 2] = cal

        cal = 0
        for j in range(n - l):
            cal += (z_values[j][0] * z_values[j+1][1] / (n - l))
        ACCN[(l-1)*9 + 3] = cal

        cal = 0
        for j in range(n - l):
            cal += (z_values[j][0] * z_values[j+1][2] / (n - l))
        ACCN[(l-1)*9 + 4] = cal

        cal = 0
        for j in range(n - l):
            cal += (z_values[j][1] * z_values[j+1][0] / (n - l))
        ACCN[(l-1)*9 + 5] = cal

        cal = 0
        for j in range(n - l):
            cal += (z_values[j][1] * z_values[j+1][2] / (n - l))
        ACCN[(l-1)*9 + 6] = cal

        cal = 0
        for j in range(n - l):
            cal += (z_values[j][2] * z_values[j+1][0] / (n - l))
        ACCN[(l-1)*9 + 7] = cal

        cal = 0
        for j in range(n - l):
            cal += (z_values[j][2] * z_values[j+1][1] / (n - l))
        ACCN[(l-1)*9 + 8] = cal

    return ACCN


In [ ]:
positive = pd.read_csv("/content/drive/MyDrive/Postdoctoral research/epipred/v2/transformed_positive.txt",
                        sep = "\t", header = None)

negative = pd.read_csv("/content/drive/MyDrive/Postdoctoral research/epipred/v2/transformed_negative.txt",
                        sep = "\t", header = None)

data = pd.concat([positive, negative], axis = 0)

x = data.iloc[:, 0:45].values
y = data.iloc[:, 45].values
scaler = StandardScaler()

x = scaler.fit_transform(x)

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(50, activation='relu', input_shape=(45,)))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
np.random.seed(42)

n_folds = 10

skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

fold_train_metrics = []
fold_test_metrics = []


for i, (train_index, test_index) in enumerate(skf.split(x, y)):
    print("Fold", i+1)

    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    tf.random.set_seed(42)

    model.fit(x_train, y_train, epochs=500, batch_size=32, verbose=0)

    y_train_pred = model.predict(x_train)
    y_train_pred = np.where(y_train_pred >= best_threshold, 1, 0)
    train_acc = accuracy_score(y_train, y_train_pred)
    tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
    train_spec = tn / (tn + fp)
    train_recall = recall_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred)
    train_loss = model.evaluate(x_train, y_train, verbose=0)[0]

    y_test_pred = model.predict(x_test)
    y_test_pred = np.where(y_test_pred >= best_threshold, 1, 0)
    test_acc = accuracy_score(y_test, y_test_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
    test_spec = tn / (tn + fp)
    test_recall = recall_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)
    test_loss = model.evaluate(x_test, y_test, verbose=0)[0]

    fold_train_metrics.append([train_acc, train_spec, train_recall, train_precision, train_loss])
    fold_test_metrics.append([test_acc, test_spec, test_recall, test_precision, test_loss])


metrics_df = pd.DataFrame({
    'Fold': np.arange(1, n_folds+1),
    'Train Accuracy': [metrics[0] for metrics in fold_train_metrics],
    'Train Specificity': [metrics[1] for metrics in fold_train_metrics],
    'Train Recall': [metrics[2] for metrics in fold_train_metrics],
    'Train Precision': [metrics[3] for metrics in fold_train_metrics],
    'Train Loss': [metrics[4] for metrics in fold_train_metrics],
    'Test Accuracy': [metrics[0] for metrics in fold_test_metrics],
    'Test Specificity': [metrics[1] for metrics in fold_test_metrics],
    'Test Recall': [metrics[2] for metrics in fold_test_metrics],
    'Test Precision': [metrics[3] for metrics in fold_test_metrics],
    'Test Loss': [metrics[4] for metrics in fold_test_metrics]
})

metrics_df.to_csv('/content/drive/MyDrive/Postdoctoral research/epipred/metrics/antigen/1_10_model_metrics.csv', index=False)

In [ ]:
np.random.seed(42)

n_folds = 10

skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

interp_tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

colors = plt.cm.get_cmap('Set1', n_folds)

for i, (train_index, test_index) in enumerate(skf.split(x, y)):
    print("Fold", i+1)

    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    tf.random.set_seed(42)

    model.fit(x_train, y_train, epochs=500, batch_size=32, verbose=0)

    y_pred_prob = model.predict(x_test)
    y_pred_prob_positive = y_pred_prob[:, 0]

    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob_positive)

    interp_tpr = np.interp(mean_fpr, fpr, tpr)
    interp_tpr[0] = 0.0
    interp_tprs.append(interp_tpr)

    auc = roc_auc_score(y_test, y_pred_prob_positive)
    aucs.append(auc)

    plt.plot(fpr, tpr, color=colors(i), label='Fold {} (AUC = {:.2f})'.format(i+1, auc))



mean_tpr = np.mean(interp_tprs, axis=0)
mean_auc = np.mean(aucs)
plt.plot([0, 1], [0, 1], 'k--', label='Random (AUC = 0.50)')
plt.plot(mean_fpr, mean_tpr, color='b', label='Mean ROC (AUC = {:.2f})'.format(mean_auc))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - 10-fold Cross Validation')
plt.legend(loc='lower right')
plt.show()

In [ ]:
data_epi = pd.read_csv("/content/drive/MyDrive/Postdoctoral research/epipred/dataset_ort.csv")

x_epi = data_epi.iloc[:, 0:45].values
y_epi = data_epi.iloc[:, 46].values

In [ ]:
model_epi = keras.Sequential()
model_epi.add(layers.Dense(50, activation='relu', input_shape=(45,)))
model_epi.add(layers.Dense(50, activation='relu'))
model_epi.add(layers.Dense(50, activation='relu'))
model_epi.add(layers.Dense(1, activation='sigmoid'))

model_epi.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
np.random.seed(42)

n_folds = 10

interp_tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

colors = plt.cm.get_cmap('Set1', n_folds)

for i, (train_index, test_index) in enumerate(skf.split(x_epi, y_epi)):
    print("Fold", i+1)

    x_train, x_test = x_epi[train_index], x_epi[test_index]
    y_train, y_test = y_epi[train_index], y_epi[test_index]

    tf.random.set_seed(42)

    model_epi.fit(x_train, y_train, epochs=100, batch_size=32, verbose=0)

    y_pred_prob = model_epi.predict(x_test)
    y_pred_prob_positive = y_pred_prob[:, 0]





    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob_positive)

    interp_tpr = np.interp(mean_fpr, fpr, tpr)
    interp_tpr[0] = 0.0
    interp_tprs.append(interp_tpr)

    auc = roc_auc_score(y_test, y_pred_prob_positive)
    aucs.append(auc)

    plt.plot(fpr, tpr, color=colors(i), label='Fold {} (AUC = {:.2f})'.format(i+1, auc))

mean_tpr = np.mean(interp_tprs, axis=0)
mean_auc = np.mean(aucs)
plt.plot([0, 1], [0, 1], 'k--', label='Random (AUC = 0.50)')
plt.plot(mean_fpr, mean_tpr, color='b', label='Mean ROC (AUC = {:.2f})'.format(mean_auc))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - 10-fold Cross Validation')
plt.legend(loc='lower right')
plt.show()

Now I start going over the new data to predict

In [ ]:
path_proteome = "/content/drive/MyDrive/Postdoctoral research/epipred/poxviruses_proteome/"

for item in os.listdir(path_proteome):
  if '.fasta' in item:
    total_proteins = 0
    total_antigens = 0
    item_new = re.sub(r"\.fasta$", ".csv", item)

    with open("/content/drive/MyDrive/Postdoctoral research/epipred/poxipred/"+item_new, 'a+') as out:
      out.write("Description,Sequence,Epitope_1, Epitope_2, Epitope_3, Epitope_4, Epitope_5\n")

    with open(path_proteome+item, "r") as file:
      for record in SeqIO.parse(file, "fasta"):
        protein_coded = ACC(record.seq)
        protein_coded = np.array(protein_coded)
        protein_coded = protein_coded.reshape(1, -1)
        pred = model.predict(protein_coded)
        total_proteins+=1

        if (pred>=0.5):### If a protein passes this if, it is a potential antigen, now find epitopes for it
          with open("/content/drive/MyDrive/Postdoctoral research/epipred/poxipred/"+item_new, 'a+') as out:
            out.write(str(record.description)+","+str(record.seq)+",")

          total_antigens+=1
          print(record.seq)
          start_pos = 5
          if len(record.seq) > 20:###smal proteins wont work

            end_pos = len(record.seq)
            n_epitopes = 0
            size_epitope_list = [8,9,10,11,12,13,14]
            attempts = 0
            size_epitope = random.choice(size_epitope_list)
            max_attempts = end_pos//size_epitope
            list_epitope = []


            while attempts < max_attempts and n_epitopes<5:
              attempts+=1
              epitope_position_start = random.randint(start_pos, end_pos-size_epitope)
              epitope = record.seq[epitope_position_start:epitope_position_start+size_epitope]
              epitope_transformed = ACC(epitope)
              epitope_transformed = np.array(epitope_transformed)
              epitope_transformed = epitope_transformed.reshape(1, -1)
              epitope_prediction = model_epi.predict(epitope_transformed)
              list_epitope.append(epitope)

              if epitope_prediction >= 0.5:
                print(epitope)
                n_epitopes+=1
                size_epitope = random.choice(size_epitope_list)
                list_epitope.append(epitope)

                with open("/content/drive/MyDrive/Postdoctoral research/epipred/poxipred/"+item_new, 'a+') as out:
                  out.write(str(epitope)+",")
                  epitope = ""

            with open("/content/drive/MyDrive/Postdoctoral research/epipred/poxipred/"+item_new, 'a+') as out:
              out.write(str(epitope)+"\n")
              epitope = ""

      with open ("/content/drive/MyDrive/Postdoctoral research/epipred/poxipred/antigen_count.txt", 'a+') as out2:
        out2.write(item+"\t"+str(total_proteins)+"\t"+str(total_antigens)+"\n")